In [5]:
import pandas as pd
import numpy as np
# from sklearn import feature_selection

# # Visualization
# import matplotlib.pyplot as plt
# import seaborn as sns


In [7]:
application = pd.read_csv('./src/application_train.csv')

In [8]:
previous_application = pd.read_csv('./src/previous_application.csv')

In [ ]:
prev_count_inputs = [
    'NAME_CONTRACT_TYPE',
    'WEEKDAY_APPR_PROCESS_START',
    'FLAG_LAST_APPL_PER_CONTRACT',
    'NFLAG_LAST_APPL_IN_DAY',
    'NAME_CASH_LOAN_PURPOSE',
    'NAME_CONTRACT_STATUS',
    'NAME_PAYMENT_TYPE',
    'CODE_REJECT_REASON',
    'NAME_TYPE_SUITE',
    'NAME_CLIENT_TYPE',
    'NAME_GOODS_CATEGORY',
    'NAME_PORTFOLIO',
    'NAME_PRODUCT_TYPE',
    'CHANNEL_TYPE',
    'NAME_SELLER_INDUSTRY',
    'NAME_YIELD_GROUP',
    'PRODUCT_COMBINATION',
    'NFLAG_INSURED_ON_APPROVAL'
]

In [87]:
def prev_count(previous_application, col):
    # Count the number of each type of [col] information for loans previously applied
    dum = pd.get_dummies(previous_application[col]).astype(int)
    dum.columns = ['PREV_' + str(col) + '_' + str(col_name) for col_name in dum.columns]

    return previous_application.loc[:, ['SK_ID_CURR', ]].join(dum).groupby('SK_ID_CURR').sum().reindex(application['SK_ID_CURR']).fillna(0)

In [ ]:
prev_sum_inputs = [
    'AMT_ANNUITY',
    'AMT_APPLICATION',
    'AMT_CREDIT',
    'AMT_DOWN_PAYMENT'
    'AMT_GOODS_PRICE'
]

In [32]:
def prev_sum(previous_application, col):
    # Total amount of [col] information for loans previously applied
    dum = previous_application.loc[:, [col]]
    dum.columns = ['PREV_' + str(col_name) for col_name in dum.columns]
    return previous_application.loc[:, ['SK_ID_CURR', ]].join(dum).groupby('SK_ID_CURR').sum().reindex(application['SK_ID_CURR']).fillna(0)
    # return previous_application.groupby('SK_ID_CURR')[col].sum().reindex(application['SK_ID_CURR']).fillna(0).rename('PREV_' + str(col))

In [82]:
def prev_hour_process(previous_application):
    # Count the number processes started during each quadrant of the day for loans previously applied
    dum = pd.get_dummies(previous_application['HOUR_APPR_PROCESS_START']).astype(int)
    first_quad = np.arange(0, 6)
    second_quad = np.arange(6, 12)
    third_quad = np.arange(12, 18)
    fourth_quad = np.arange(18, 24)
    dum['HOUR_APPR_PROCESS_START_FIRST_QUAD'] = 0
    dum['HOUR_APPR_PROCESS_START_SECOND_QUAD'] = 0
    dum['HOUR_APPR_PROCESS_START_THIRD_QUAD'] = 0
    dum['HOUR_APPR_PROCESS_START_FOURTH_QUAD'] = 0
    for hour in first_quad:
        dum['HOUR_APPR_PROCESS_START_FIRST_QUAD'] += dum[hour]
    for hour in second_quad:
        dum['HOUR_APPR_PROCESS_START_SECOND_QUAD'] += dum[hour]
    for hour in third_quad:
        dum['HOUR_APPR_PROCESS_START_THIRD_QUAD'] += dum[hour]
    for hour in fourth_quad:
        dum['HOUR_APPR_PROCESS_START_FOURTH_QUAD'] += dum[hour]
    dum = dum.loc[:, ['HOUR_APPR_PROCESS_START_FIRST_QUAD', 'HOUR_APPR_PROCESS_START_SECOND_QUAD', 'HOUR_APPR_PROCESS_START_THIRD_QUAD', 'HOUR_APPR_PROCESS_START_FOURTH_QUAD']]
    dum.columns = ['PREV_' + str(col_name) for col_name in dum.columns]

    return previous_application.loc[:, ['SK_ID_CURR', ]].join(dum).groupby('SK_ID_CURR').sum().reindex(application['SK_ID_CURR']).fillna(0)

In [122]:
def prev_cnt_payment(previous_application):
    # Count the number of previous loans that had a certain term (in number of years)
    dum = pd.get_dummies(previous_application['CNT_PAYMENT']).astype(int)
    one_yr = np.arange(0, 12)
    two_yr = np.arange(12, 24)
    three_yr = np.arange(24, 36)
    four_yr = np.arange(36, 48)
    five_yr = np.arange(48, 60)
    six_yr = np.arange(60, 72)
    seven_yr = np.arange(72, 85)
    terms = [one_yr, two_yr, three_yr, four_yr, five_yr, six_yr, seven_yr]
    for i in range(len(terms)):
        dum['CNT_PAYMENT_{}_YR'.format(i+1)] = 0
        for month in terms[i]:
            if month in dum:
                dum['CNT_PAYMENT_{}_YR'.format(i+1)] += dum[month]

    dum = dum.loc[:, ['CNT_PAYMENT_1_YR', 'CNT_PAYMENT_2_YR', 'CNT_PAYMENT_3_YR', 'CNT_PAYMENT_4_YR', 'CNT_PAYMENT_5_YR', 'CNT_PAYMENT_6_YR', 'CNT_PAYMENT_7_YR', ]]
    dum.columns = ['PREV_' + str(col_name) for col_name in dum.columns]

    return previous_application.loc[:, ['SK_ID_CURR', ]].join(dum).groupby('SK_ID_CURR').sum().reindex(application['SK_ID_CURR']).fillna(0)

In [ ]:
prev_avg_inputs = [
    'RATE_DOWN_PAYMENT',
    'RATE_INTEREST_PRIMARY',
    'RATE_INTEREST_PRIVILEGED'
    'SELLERPLACE_AREA'
]

In [92]:
def prev_avg(previous_application, col):
    # Average of [col] information for loans previously applied
    dum = previous_application.loc[:, [col]]
    dum.columns = ['PREV_' + str(col_name) for col_name in dum.columns]
    return previous_application.loc[:, ['SK_ID_CURR', ]].join(dum).groupby('SK_ID_CURR').mean().reindex(application['SK_ID_CURR']).fillna(0)

In [ ]:
pre_earliest_latest_inputs = [
    'DAYS_DECISION',
    'DAYS_FIRST_DRAWING',
    'DAYS_FIRST_DUE',
    'DAYS_LAST_DUE_1ST_VERSION',
    'DAYS_LAST_DUE',
    'DAYS_TERMINATION'
]

In [105]:
def earliest_starting(previous_application, col):
    dum = previous_application.loc[:, [col]]
    dum.columns = ['PREV_' + str(col_name) + '_FIRST' for col_name in dum.columns]
    return previous_application.loc[:, ['SK_ID_CURR', ]].join(dum).groupby('SK_ID_CURR').min().abs().reindex(application['SK_ID_CURR']).fillna(0)

def latest_starting(previous_application, col):
    dum = previous_application.loc[:, [col]]
    dum.columns = ['PREV_' + str(col_name) + '_LAST' for col_name in dum.columns]
    return previous_application.loc[:, ['SK_ID_CURR', ]].join(dum).groupby('SK_ID_CURR').max().abs().reindex(application['SK_ID_CURR']).fillna(0)
